### Duckweed Aspiration
Test notebook for aspirating duckweed

In [1]:
# import required modules
from utils.MachineUtils import *
import time
import random
import numpy as np
import cv2 as cv2
from utils.PlatePositionUtils import *
from utils.CameraUtils import *

In [2]:
# Setup your machine connection
# List available ports in thie cell

ports = serial.tools.list_ports.comports()
print([port.name for port in ports]) 

['ttyACM0', 'ttyAMA0']


In [3]:
# Choose the correct port from above and establish connection with machine
port = '/dev/ttyACM0'
m = MachineCommunication(port)

In [4]:
# load in the relevant calibration files
# read in the calibration file
# with open("/home/pi/autofocus-test/JubileeAutofocus/camera_cal_z_110_update.json") as f:
with open("/home/pi/autofocus-test/JubileeAutofocus/camera_cal_z_60_tape.json") as f:
    cal = json.load(f)
matrix = np.array(cal['transform'])
size = cal['resolution']
print(matrix)
print(size)

m.transform = matrix
m.img_size = size

[[ 2.83177792e-01 -1.52038853e+00]
 [ 2.96527139e+00 -3.34144830e+00]
 [-4.89224219e-01  2.59932583e-01]
 [ 5.55483812e-01  7.52317823e+01]
 [-7.49778260e+01 -4.26956240e-02]
 [ 8.02453667e+01  2.39263142e+02]]
[1200, 1200]


In [24]:
# grab the camera, move the relevant height & position to take pic of duckweed
m.toolChange(1)
o = [80, 240]
m.moveTo(x=o[0], y=o[1])
# m.moveTo(z=60)
# move down more based on surface height
m.moveTo(z=65)

In [25]:
#focus camera
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

# draw a circle in the center of the frame
center = None
while center is None:
    # the first frame grab is sometimes empty
    ret, frame = cap.read()
    h, w = frame.shape[0:2]
    center = (int(w/2), int(h/2))
    print(center)

while True:
    ret, frame = cap.read()
    target = cv2.circle(frame, center, 5, (0,255,0), -1)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

(320, 240)


In [26]:
# take a picture, and take only green pixels:
f = getFrame()
showFrame(f, grid=True)
saveFrame(f, 'petri-test.jpg')

In [ ]:
m = cv2.imread('petri.jpg')
for i in range(im.shape[0]): #im.shape[0] is the y-axis
    for j in range(im.shape[1]): #im.shape[1] is the x returns X and Y axes of the image
        if im[i, j][0] > 0.9*im[i, j][1]:  #If more blue than green in pixel than set to black.         
            im[i, j] = [0, 0, 0]
        if np.array([x < 90 for x in im[i, j]]).all():
            im[i, j] = [0, 0, 0]
showFrame(im)

In [ ]:
# now we want to pick a random green pixel-- this is the duckweed to pick up
r = 200 # look in a 200 mm circle around the center
px = None

while px is None:
    randx = random.randint(-r, r)
    randy = random.randint(-r, r)
    coord = [600 + randx, 600 + randy]
    if im[coord[0], coord[1]][1] > 0: # if green
        px = coord
    else:
        continue

print(px)

In [13]:
# convert px coord to real coord
def px_to_real(x,y, absolute = False):
        x = (x / m.img_size[0]) - 0.5
        y = (y / m.img_size[1]) - 0.5
        a = 1 if absolute else 0

        return (m.transform.T @ np.array([x**2, y**2, x * y, x, y, a]))

In [28]:
# alternatively, manually click a frond
im = cv2.imread('petri-test.jpg')
# px = selectPoint(im)
pts = selectPoint(im, num_pts=12)

[(486.6255911718338, 511.5336311087757), (929.8355603933637, 630.4436228511373), (692.0155769086405, 612.426957435628), (666.7922453269273, 769.1719465505593), (951.455558891975, 832.230275504842), (520.8572554613015, 879.0736055851662), (329.8806020569025, 882.6769386682681), (618.1472487050521, 190.8369867127094)]


In [29]:
# convert px coord to real coord
def px_to_real(x,y, absolute = False):
        x = (x / m.img_size[0]) - 0.5
        y = (y / m.img_size[1]) - 0.5
        a = 1 if absolute else 0

        return (m.transform.T @ np.array([x**2, y**2, x * y, x, y, a]))

fronds = []
for pt in pts:
    frond_off = px_to_real(pt[0], pt[1])
    frond = [o[0] - frond_off[0], o[1] - frond_off[1]]
    fronds.append(frond)

In [30]:
# print(px)
# frond_off = px_to_real(px[0], px[1])
# frond = [o[0] - frond_off[0], o[1] - frond_off[1]]
# print(frond)
for f in fronds:
    print(f)

[74.50973180356098, 247.13457282174033]
[81.72959090991758, 219.43785515794124]
[80.73226636693782, 234.24078707308126]
[90.48323082498254, 235.88768262277003]
[94.23979418048287, 218.2152336342473]
[97.30446748575547, 245.1629590167579]
[97.5822902940912, 257.2209332733764]
[54.07914212185028, 239.23789861777004]


In [37]:
# switch tools, account for offset
m.toolChange(2)
syringe_off = [-5.5, 3.3]
surface_depth = -61.2 # z
syringe_zero = 71.7
print(syringe_zero + surface_depth)
# m.move(dz=syringe_zero + surface_depth)
# m.moveTo(m.moveTo(x=frond[0] + syringe_off[0], y=frond[1] + syringe_off[1]))

10.5


In [43]:
# aspirate duckweed
# first move to surface
def aspirate():
    m.moveTo(z=surface_depth)
    m.dwell(1000)
    m.move(dz=-0.5) # press slightly
    m.dwell(1000)
    m.move(dz=5, de=50, s=1800) # aspirate!

def dispense():
    m.move(dz=10)
    m.dwell(1000)
    m.move(de=-50, s=1500)
    m.dwell(500)
    
for frond in fronds:
    m.moveTo(m.moveTo(x=frond[0] + syringe_off[0], y=frond[1] + syringe_off[1]))
    aspirate()
    m.dwell(3000)
    dispense()
    

G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1   Z10.00  F6000.00
G1    E-50.00 F1500.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1   Z10.00  F6000.00
G1    E-50.00 F1500.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1   Z10.00  F6000.00
G1    E-50.00 F1500.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1   Z10.00  F6000.00
G1    E-50.00 F1500.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1   Z10.00  F6000.00
G1    E-50.00 F1500.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1   Z10.00  F6000.00
G1    E-50.00 F1500.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1   Z10.00  F6000.00
G1    E-50.00 F1500.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1   Z10.00  F6000.00
G1    E-50.00 F1500.00


In [35]:
# move camera over to check if it is centered
check = 2
m.moveTo(x=fronds[check][0], y=fronds[check][1])

In [36]:
showFrame(getFrame(), grid=True)

In [ ]:
m.setExtruderRelative()

In [ ]:
# set the height of the surface of the water
# by stepping slowly in z
surface_depth = -61.6 # z


In [5]:
# set the x,y location of the center of the petri dish
petri_location = [77, 237]

In [6]:
m.moveTo(x=petri_location[0], y=petri_location[1])

In [ ]:
# aspirate duckweed
# first move to surface
m.moveTo(z=surface_depth)
m.dwell(1000)
m.move(dz=-0.5) # press slightly
m.dwell(1000)
m.move(dz=5, de=50, s=2000) # aspirate!

In [150]:
m.move(dz=5, de=30, s=1800)

G1   Z5.00 E30.00 F1800.00


In [181]:
m.move(de=-25, s=1500)


G1    E-25.00 F1500.00


In [ ]:
# move to a random point to pick up duckweed
# assumes dense petri dish
r = 25
rx = r * random.random()
ry = r * random.random()

# move in xy first
m.moveTo(x=petri_location[0] + rx, y=petri_location[1] + ry)

# aspirate duckweed
# first move to surface
m.moveTo(z=surface_depth)
time.sleep(3) # wait a bit
m.move(dz=-1) # press slightly
time.sleep(3) # wait a bit
m.move(dz=5, de=20, s=2000) # aspirate!

In [ ]:
# dipense
m.move(dz=8)
m.move(de=-20, s=1500)

In [65]:
m.setExtruderRelative()
m.move(dz=-5, de=30, s=1800) # aspirate!

G1   Z-5.00 E30.00 F1800.00


In [67]:
m.move(de=-50, s=1000) # aspirate!

G1    E-50.00 F1000.00


In [46]:
m.move(de=10, s=1000)

G1    E10.00 F1000.00


In [ ]:
# without random movement
# aspirate duckweed
# first move to surface
m.moveTo(z=surface_depth)
time.sleep(5) # wait a bit
m.move(dz=-0.75) # press slightly
time.sleep(0.75) # wait a bit
m.move(dz=10, de=20, s=2000) # aspirate!

In [42]:
m.move(de=25, s=1800)

G1    E25.00 F1800.00


In [23]:
# debug syringe
# aspirate duckweed
# first move to surface
def aspirate():
    # m.moveTo(z=surface_depth)
    m.dwell(1000)
    m.move(dz=-0.5) # press slightly
    m.dwell(1000)
    m.move(dz=5, de=50, s=1800) # aspirate!

def dispense():
    # m.move(dz=10)
    m.dwell(500)
    m.move(de=-50, s=2000)
    m.dwell(500)
    
for frond in fronds:
    m.moveTo(m.moveTo(x=frond[0] + syringe_off[0], y=frond[1] + syringe_off[1]))
    aspirate()
    m.dwell(3000)
    dispense()

G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1    E-50.00 F2000.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1    E-50.00 F2000.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1    E-50.00 F2000.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1    E-50.00 F2000.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1    E-50.00 F2000.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1    E-50.00 F2000.00
G1   Z-0.50  F6000.00
G1   Z5.00 E50.00 F1800.00
G1    E-50.00 F2000.00


KeyboardInterrupt: 

In [7]:
o = [77, 200]

In [21]:
surface_depth = -26.3

In [37]:
# move to a random point to pick up duckweed
# assumes dense petri dish
for i in range(10):
    r = 10
    rx = random.randint(-r, r)
    ry = random.randint(-r, r)

    # move in xy first
    m.moveTo(x=o[0] + rx, y=o[1] + ry)

    # aspirate duckweed
    # first move to surface
    m.moveTo(z=surface_depth)
    m.dwell(1000) # wait a bit
    m.move(dz=-0.5) # press slightly
    m.dwell(1000)
    m.move(de=20, s=2000) # aspirate!
    m.move(dz=5, s=500) # aspirate!

    m.dwell(1000)
    m.move(de=-20, s=2000) # dispense

# m.move(dz=5, de=20, s=2000) # aspirate!

G1   Z-0.50  F6000.00
G1    E20.00 F2000.00
G1   Z5.00  F500.00
G1    E-20.00 F2000.00
G1   Z-0.50  F6000.00
G1    E20.00 F2000.00
G1   Z5.00  F500.00
G1    E-20.00 F2000.00
G1   Z-0.50  F6000.00
G1    E20.00 F2000.00
G1   Z5.00  F500.00
G1    E-20.00 F2000.00
G1   Z-0.50  F6000.00
G1    E20.00 F2000.00
G1   Z5.00  F500.00
G1    E-20.00 F2000.00
G1   Z-0.50  F6000.00
G1    E20.00 F2000.00
G1   Z5.00  F500.00
G1    E-20.00 F2000.00
G1   Z-0.50  F6000.00
G1    E20.00 F2000.00
G1   Z5.00  F500.00
G1    E-20.00 F2000.00
G1   Z-0.50  F6000.00
G1    E20.00 F2000.00
G1   Z5.00  F500.00
G1    E-20.00 F2000.00
G1   Z-0.50  F6000.00
G1    E20.00 F2000.00
G1   Z5.00  F500.00
G1    E-20.00 F2000.00
G1   Z-0.50  F6000.00
G1    E20.00 F2000.00
G1   Z5.00  F500.00
G1    E-20.00 F2000.00
G1   Z-0.50  F6000.00
G1    E20.00 F2000.00
G1   Z5.00  F500.00
G1    E-20.00 F2000.00


In [36]:
m.move(de=-20, s=2000) # dispense

G1    E-20.00 F2000.00
